# main.ipynb
A notebook designing the substrate for the matching system

In [55]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [56]:
import pyphi
from substrate_modeler import unit, substrate

from tqdm.auto import tqdm
import numpy as np
import pickle

In [57]:
pyphi.config.PARTITION_TYPE = 'TRI'
pyphi.config.PROGRESS_BARS = False
OUTPUT_DIR = 'data/for_matteo/'

In [58]:
labels = ["A", "B", "C", "D", "E", "F", "G", "H"] 
N = len(labels)

# For a time grid

In [72]:
# parameters for sigmoidal subunit

# V1_determinism = 4
V1_determinism = 1

V1_threshold = 0.0
V1_floor = 0.00
V1_ceiling = 1.00

# define mapping to scale weight based on (unit_state, input_state)
V1_weight_scale_mapping = {
    (0, 0): 1.0,
    (1, 0): 0.5,
    (0, 1): 0.75,
    (1, 1): 1.5
}

self_loop = 0.3
lateral = 0.6
second = -0.1
w = [
    self_loop,  # self
    lateral,  # near neighbor
    second
]
V1_weights = [
    w,w,w,w,w,w,
    [
        self_loop,  # self
        lateral,  # near neighbor
    ],
    [
        self_loop,  # self
    ],
]

In [73]:

V1_sigmoid_inputs = [
    (0,1,2),
    (1,2,3),
    (2,3,4),
    (3,4,5),
    (4,5,6),
    (5,6,7),
    (6,7,),
    (7,),
]

V1 = [
    unit.Unit(
        index=i,
        label=labels[i],
        state=(0,),
        inputs=V1_sigmoid_inputs[i],
        input_state=(0,)*len(V1_weights[i]),
        mechanism="resonnator",
        params=dict(
            input_weights=V1_weights[i],
            determinism=V1_determinism,
            threshold=V1_threshold,
            weight_scale_mapping=V1_weight_scale_mapping,
            floor=V1_floor,
            ceiling=V1_ceiling,
        ),
    )
    for i in range(N)
] 

# All states

In [89]:
substrate_states = [
    (0,0,0,0,0,0,0,0,),
    (0,0,1,0,0,0,0,0,),
    (0,0,1,0,1,0,0,0,),
    (0,0,1,1,0,0,0,0,),
    (1,1,1,1,1,0,0,0,),
    (1,1,1,1,1,1,1,1,),
]

# create subsystems for all states

In [90]:
systems = {
    state:
    substrate.Substrate(
        V1, state, state, implicit=True
    ).subsystem()
    for state in tqdm(substrate_states)
}


  0%|          | 0/6 [00:00<?, ?it/s]

In [91]:
with open(OUTPUT_DIR+'systems_time.pkl','wb') as f:
    pickle.dump(systems,f)
    

In [92]:
def get_concepts(
    subsystem,
    mechanisms,
    purviews=False,
    cause_purviews=False,
    effect_purviews=False
):
    if not purviews:
        purviews = [purviews]*len(mechanisms)
    if not cause_purviews:
        cause_purviews = [cause_purviews]*len(mechanisms)
    if not effect_purviews:
        effect_purviews = [effect_purviews]*len(mechanisms)
        
    return [
        subsystem.concept(
            mechanism,
            purview,
            cause_purview,
            effect_purview
        )
        for (
            mechanism,
            purview,
            cause_purview,
            effect_purview
        ) in tqdm(
            zip(mechanisms,purviews,cause_purviews,effect_purviews),
            total=len(mechanisms)
        )
    ]

In [93]:
mechanisms= [
    (1,),(3,),(5,),(7,),
    (1,2),(3,4),(5,6),
    (1,2,3),(4,5,6),
    (1,2,3,4),(4,5,6,7),
    (1,2,3,4,5)
]

In [94]:
concepts = {
    state:
    get_concepts(system, mechanisms)
    for state, system in tqdm(systems.items())
}

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [95]:
df_time = [
    ces2df(CES(cons, subsystem=system), node_labels=system.node_labels)
    for cons, system in zip(concepts.values(),systems.values())
]

In [102]:
df_time[2]

,mechanism,mechanism_state,phi,cause_purview,cause_state,cause_phi,effect_purview,effect_state,effect_phi
0,B|,0,0.171652,C|,1,0.171652,A|,0,0.232478
1,D|,0,0.124151,E|,1,0.171652,C|D,10,0.124151
2,F|,0,0.121758,G|,0,0.232235,E|F,10,0.121758
3,H|,0,0.115023,H|,0,0.115023,G|,0,0.232790
4,B|C,01,0.107752,C|,1,0.198856,B|,0,0.107752
5,D|E,01,0.107752,E|,1,0.199479,D|,0,0.107752
6,F|G,00,0.103325,G|,0,0.129734,F|,0,0.103325
7,B|C|D,010,0.043854,C|D,10,0.043854,B|C,01,0.049635
8,E|F|G,100,0.049811,F|G,00,0.049811,E|F,10,0.057964
9,B|C|D|E,0101,0.008978,C|D|E,101,0.008978,B|C|D,010,0.011579
